In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, r2_score # Import r2_score here

# Select columns for the analysis
data = drone_df_edited[['time', 'battery_current', 'battery_voltage', 'payload', 'altitude', 'speed', 'wind_speed', 'wind_angle']]

# Define dependent (y = altitude) and independent (X) variables
X = data[['battery_current', 'battery_voltage', 'payload', 'speed', 'wind_speed', 'wind_angle']]
y = data[['altitude']]  # Target variable

# Scaling using MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)


# Create a datset with lookback (25)
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), :-1]  # For independent variables
        X.append(a)
        Y.append(dataset[i + look_back, -1])  # For dependent variable (altitude)
    return np.array(X), np.array(Y)

look_back = 25  # Number of previous steps to look at
X, y = create_dataset(np.concatenate([X, y], axis=1), look_back)

# Reshape input 
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=100))
model.add(Dense(units=1))  # Use an output layer with 1 neuron for altitude

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=32)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Invert the scaling to get actual altitude values
y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1))
y_test = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# print out MSE and R-squared values
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")